In [1]:
# File: report_lookups.py

import csv

# ---------------------------------------------------------------------------
# ADJUST THESE IMPORTS TO MATCH YOUR OWN FOLDER/FILE STRUCTURE
# ---------------------------------------------------------------------------
# Example (assuming you have your code in packages named exactly "DHW", "Elec", etc.):
# from DHW.dhw_lookup import dhw_lookup
# from Elec.lighting_lookup import lighting_lookup
# from ventilation.ventilation_lookup import ventilation_lookup
# from setzone.zone_sizing_lookup import zone_sizing_lookup
# from tempground.groundtemp_lookup import groundtemp_lookup

# If your code is not set up as Python packages, you might do:
# from .dhw_lookup import dhw_lookup
# or
# import sys
# sys.path.append("D:/Documents/E_Plus_2026_py/DHW")
# from dhw_lookup import dhw_lookup
#
# For now, let's assume you can just import them directly:
try:
    from DHW.dhw_lookup import dhw_lookup
    from Elec.lighting_lookup import lighting_lookup
    from ventilation.ventilation_lookup import ventilation_lookup
    from setzone.zone_sizing_lookup import zone_sizing_lookup
    from tempground.groundtemp_lookup import groundtemp_lookup
except ImportError:
    # If you can't import them like this, comment these out and adapt as needed
    print("[WARNING] Could not import from your modules. Please adjust imports!")
    dhw_lookup = {}
    lighting_lookup = {}
    ventilation_lookup = {}
    zone_sizing_lookup = {}
    groundtemp_lookup = {}

# ---------------------------------------------------------------------------
# REPORTING FUNCTIONS - each function focuses on one dictionary structure
# ---------------------------------------------------------------------------

def report_dhw_lookup(writer):
    """
    Example for dhw_lookup, which might have the structure:
      dhw_lookup = {
        "pre_calibration": {
            "Residential_SingleFamily_Small": {
                "occupant_density_m2_per_person_range": (None, None),
                "liters_per_person_per_day_range": (45.0, 55.0),
                ...
            },
            "Office": { ... }
        },
        "post_calibration": { ... }
      }

    We'll write rows with columns:
      [lookup_name, stage, bldg_key, param_name, min_val, max_val]
    """
    lookup_name = "dhw_lookup"
    for stage, stage_dict in dhw_lookup.items():
        for bldg_key, param_dict in stage_dict.items():
            for param_name, val_range in param_dict.items():
                # Some param ranges may be (None, None), or a single number/schedule
                if isinstance(val_range, tuple) and len(val_range) == 2:
                    min_val, max_val = val_range
                    writer.writerow([lookup_name, stage, bldg_key, param_name, min_val, max_val])
                else:
                    # If it's not a tuple or is a single fixed value
                    writer.writerow([lookup_name, stage, bldg_key, param_name, val_range, val_range])


def report_lighting_lookup(writer):
    """
    Example for lighting_lookup, with structure:
      lighting_lookup = {
        "pre_calibration": {
          "residential": {
            "LIGHTS_WM2_range": (0.0, 0.0),
            "PARASITIC_WM2_range": (0.0, 0.0),
            ...
          },
          "Meeting Function": {...}
        },
        "post_calibration": {...}
      }
    """
    lookup_name = "lighting_lookup"
    for stage, stage_dict in lighting_lookup.items():
        for bldg_subkey, param_dict in stage_dict.items():
            for param_name, val_range in param_dict.items():
                if isinstance(val_range, tuple) and len(val_range) == 2:
                    writer.writerow([lookup_name, stage, bldg_subkey, param_name, val_range[0], val_range[1]])
                else:
                    writer.writerow([lookup_name, stage, bldg_subkey, param_name, val_range, val_range])


def report_ventilation_lookup(writer):
    """
    ventilation_lookup often has a deeper nested structure, e.g.:
      ventilation_lookup = {
        "pre_calibration": {
          "residential_infiltration_range": { "A_corner": (1.0,1.2), ... },
          "non_res_infiltration_range": {...},
          "year_factor_range": {...},
          "system_control_range_res": {
              "A": {"f_ctrl_range": (0.9,1.0)}, ...
          },
          ...
        },
        "post_calibration": {...}
      }
    We'll walk each sub-dict carefully.
    """
    lookup_name = "ventilation_lookup"
    for stage, stage_dict in ventilation_lookup.items():
        # top-level keys (e.g. "residential_infiltration_range", "year_factor_range", etc.)
        for cat_name, cat_data in stage_dict.items():
            # cat_data might be a dict of subkeys or a direct tuple
            if isinstance(cat_data, dict):
                for subkey, val in cat_data.items():
                    # if 'val' is another dict, e.g. "system_control_range_res": { "A": {"f_ctrl_range": (0.9,1.0)}}
                    if isinstance(val, dict):
                        for param_name, param_val in val.items():
                            if isinstance(param_val, tuple) and len(param_val) == 2:
                                writer.writerow([lookup_name, stage, f"{cat_name}:{subkey}",
                                                 param_name, param_val[0], param_val[1]])
                            else:
                                writer.writerow([lookup_name, stage, f"{cat_name}:{subkey}",
                                                 param_name, param_val, param_val])
                    else:
                        # If val is a tuple for infiltration range, etc.
                        if isinstance(val, tuple) and len(val) == 2:
                            writer.writerow([lookup_name, stage, cat_name, subkey, val[0], val[1]])
                        else:
                            writer.writerow([lookup_name, stage, cat_name, subkey, val, val])
            else:
                # cat_data is not a dict => might be a single tuple or number
                if isinstance(cat_data, tuple) and len(cat_data) == 2:
                    writer.writerow([lookup_name, stage, cat_name, "", cat_data[0], cat_data[1]])
                else:
                    writer.writerow([lookup_name, stage, cat_name, "", cat_data, cat_data])


def report_zone_sizing_lookup(writer):
    """
    zone_sizing_lookup = {
      "pre_calibration": {
        "residential": {
          "cooling_supply_air_temp_range": (13.5,14.5),
          "heating_supply_air_temp_range": (48.0,52.0),
          ...
        },
        "non_residential": {...}
      },
      "post_calibration": {...}
    }
    """
    lookup_name = "zone_sizing_lookup"
    for stage, stage_dict in zone_sizing_lookup.items():
        for bldg_func, param_dict in stage_dict.items():
            for param_name, val_range in param_dict.items():
                if isinstance(val_range, tuple) and len(val_range) == 2:
                    writer.writerow([lookup_name, stage, bldg_func, param_name,
                                     val_range[0], val_range[1]])
                else:
                    writer.writerow([lookup_name, stage, bldg_func, param_name,
                                     val_range, val_range])


def report_groundtemp_lookup(writer):
    """
    groundtemp_lookup = {
      "pre_calibration": {
        "January": (2.0,3.0),
        "February": (3.5,5.0),
        ...
      },
      "post_calibration": {...}
    }
    """
    lookup_name = "groundtemp_lookup"
    for stage, month_dict in groundtemp_lookup.items():
        for month_name, val_range in month_dict.items():
            # usually a (min,max) pair
            if isinstance(val_range, tuple) and len(val_range) == 2:
                writer.writerow([lookup_name, stage, month_name, "", val_range[0], val_range[1]])
            else:
                writer.writerow([lookup_name, stage, month_name, "", val_range, val_range])


# ---------------------------------------------------------------------------
# MAIN REPORT FUNCTION
# ---------------------------------------------------------------------------
def main():
    """
    Creates a CSV file `lookup_report.csv` that merges info from multiple lookups.
    Adjust or add more 'report_*' calls as needed.
    """
    with open("lookup_report.csv", "w", newline="") as f:
        writer = csv.writer(f)
        # Write header row
        writer.writerow(["LookupName", "CalibrationStage", "KeyOrCategory", "ParamName", "MinValue", "MaxValue"])

        # Call each specialized reporting function
        report_dhw_lookup(writer)
        report_lighting_lookup(writer)
        report_ventilation_lookup(writer)
        report_zone_sizing_lookup(writer)
        report_groundtemp_lookup(writer)
        # ... Add more if you have additional lookups

    print("[INFO] Finished creating 'lookup_report.csv' with min/max (or fixed) values from all lookups.")


if __name__ == "__main__":
    main()


[INFO] Finished creating 'lookup_report.csv' with min/max (or fixed) values from all lookups.


In [3]:
import os

def export_structure_and_content(base_directory, output_file, extensions=('.py', '.csv')):
    """
    Recursively walks through base_directory, finds all files with the given extensions,
    and writes directory structure, filenames, and contents to output_file.
    
    :param base_directory: The path of the directory to walk through.
    :param output_file: The path of the text file where results will be written.
    :param extensions: A tuple of file extensions to include.
    """
    with open(output_file, 'w', encoding='utf-8') as out:
        # Walk through the directory structure
        for root, dirs, files in os.walk(base_directory):
            # Write out the current folder name
            out.write(f"Folder: {root}\n")
            
            # Filter out files by the desired extensions
            for file_name in files:
                if file_name.lower().endswith(extensions):
                    file_path = os.path.join(root, file_name)
                    
                    # Write out the file name
                    out.write(f"\n  File: {file_name}\n")
                    
                    # Read and write the file contents
                    try:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            content = f.read()
                    except UnicodeDecodeError:
                        # If there's an encoding issue, try ignoring errors or switch to a different encoding
                        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                            content = f.read()
                    
                    out.write("  --- File Contents Start ---\n")
                    out.write(content)
                    out.write("\n  --- File Contents End ---\n")
            
            out.write("\n" + "="*80 + "\n\n")


if __name__ == "__main__":
    # Example usage:
    # Change 'your_directory_path' to the path you want to scan
    # Change 'output.txt' to the desired output file name
    base_dir = "D:\Documents\E_Plus_2028_py"
    output_txt = "output.txt"
    
    export_structure_and_content(base_dir, output_txt)
    print(f"All .py and .csv files from '{base_dir}' have been exported to '{output_txt}'.")


All .py and .csv files from 'D:\Documents\E_Plus_2028_py' have been exported to 'output.txt'.


In [3]:
import os

def export_structure_and_content(base_directory, output_file, extensions=('.py', '.csv')):
    """
    Recursively walks through base_directory, finds all files with the given extensions,
    and writes directory structure, filenames, and contents to output_file.
    
    :param base_directory: The path of the directory to walk through.
    :param output_file: The path of the text file where results will be written.
    :param extensions: A tuple of file extensions to include.
    """
    with open(output_file, 'w', encoding='utf-8') as out:
        # Walk through the directory structure
        for root, dirs, files in os.walk(base_directory):
            # Write out the current folder name
            out.write(f"Folder: {root}\n")
            
            # Filter out files by the desired extensions
            for file_name in files:
                if file_name.lower().endswith(extensions):
                    file_path = os.path.join(root, file_name)
                    
                    # Write out the file name
                    out.write(f"\n  File: {file_name}\n")
                    
                    # Read and write the file contents
                    try:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            content = f.read()
                    except UnicodeDecodeError:
                        # If there's an encoding issue, try ignoring errors or switch to a different encoding
                        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                            content = f.read()
                    
                    out.write("  --- File Contents Start ---\n")
                    out.write(content)
                    out.write("\n  --- File Contents End ---\n")
            
            out.write("\n" + "="*80 + "\n\n")


if __name__ == "__main__":
    # Example usage:
    # Change 'your_directory_path' to the path you want to scan
    # Change 'output.txt' to the desired output file name
    base_dir = r"D:\Documents\E_Plus_2029_py"
    output_txt = "output.txt"
    
    export_structure_and_content(base_dir, output_txt)
    print(f"All .py and .csv files from '{base_dir}' have been exported to '{output_txt}'.")


All .py and .csv files from 'D:\Documents\E_Plus_2029_py' have been exported to 'output.txt'.


i have the following code that i have, i first made it as piot project with test data. now i want to refine the lookup tables and  input dta. so together step by step we will woork on that. my projectis mainly based on archtetype of buildings. it can be different from object to object thatwe will work on it together.

i firasst provide how my input data will be, and te code i have. you first have ann understanding and a view of what we have and will go to next step later. 



these columns from my data can be :


for functions and types and age range it can be:
1.
building_function	residential_type	non_residential_type
residential	Corner House	null
non_residential	null	Meeting Function
residential	Apartment	null
residential	Terrace or Semi-detached House
non_residential	null	Healthcare Function
non_residential	null	Sport Function
non_residential	null	Cell Function
non_residential	null	Retail Function
residential	Detached House	null
non_residential	null	Industrial Function
residential	Two-and-a-half-story House
non_residential	null	Accommodation Function
non_residential	null	Office Function
non_residential	null	Education Function
non_residential	null	Other Use Function
2.
age_range
2015 and later
1992 - 2005
1945 - 1964
1975 - 1991
1965 - 1974
2006 - 2014
< 1945




ok, first lets fo gor geomz. as yyou can see, we have:
1. default values in geometry_lookup
2. an excel file and overriding that cantotally change this lookup table at beginnging after imports. 
3. a partially temporary overridings that can happen by user config


so, if py works as i want, then lets work on Lookup expansions. 

so,, how the look up table should be. 

also, how i need to provide my excel file. 

and also how the user configs can be 







Loads defaults from dictionaries,
Optionally overrides from an Excel file once at startup,
Optionally overrides on a per-building basis via user_config_XXX lists,
Then uses that final set of parameters to build an EnergyPlus model.